In [1]:
!pip install arcgis

     |████████████████████████████████| 2.0MB 5.8MB/s 
  Using cached https://files.pythonhosted.org/packages/e4/ed/7be20815f248b0d6aae406783c2bee392640924623c4e17b50ca90c7f74d/keyring-21.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/20/73c8fa29a4ba8f8cbdb037369e6b61ac9253664a8aa0912b52169192447f/lerc-0.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/31/7b/cd66f306c31a84a53c6a3a86e296586e8664f407a6ac5b7cfe6a433aa8c4/jupyterlab-2.2.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/1f/e9cc2c3fce32e2926581f8b6905831165235464c858ba550b6e9b8ef78c3/pyshp-2.1.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/03/4b/8b9a1afde8072c4d5710d9fa91433d504325821b038e00237dc8d6d833dc/requests_ntlm-1.1.0-py2.py3-none-any.whl
  Using cached https://f

In [2]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from IPython.display import display

gis = GIS()
map1 = gis.map()
map1

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
diegogarcia = geocode("Diego Garcia")[0]
map1.extent = diegogarcia['extent']

In [4]:
lareunion = geocode("La Reunion")[0]

In [6]:
# Annotate the map by plotting Diego Garcia, and two other search locations
map1.draw(lareunion['location'], {"title": "Reunion Island", "content": "Debris found"})
map1.draw(diegogarcia['location'], {"title": "Diego Garcia", "content": "Naval Support Facility Diego Garcia"})
map1.draw([-43.5, 90.5], {"title":"Search Location", "content":"Predicted crash location"})

# Render a feature layer representing the search area
# Source: http://www.amsa.gov.au/media/incidents/images/DIGO_00718_01_14.jpg
map1.add_layer({"type":"FeatureLayer", 
                "url" : "http://services.arcgis.com/WQ9KVmV6xGGMnCiQ/arcgis/rest/services/MH370Search/FeatureServer/1"})

In [7]:
mh370items = gis.content.search("owner:bflanagan_bureau tags:mh370", "feature service", max_items=7)
for item in mh370items:
    display(item)

<Item title:"Arc" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"MH370Search" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"mh370debris" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"UnderwaterSearch" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"SatelliteLocation" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"ActualFlightRoute" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"MH370_Flight_Path" type:Feature Layer Collection owner:bflanagan_bureau>

In [8]:
# The order of items might change, change the item number according to the order of your notebook
map1.add_layer(mh370items[6]) # MH370 Flight Path
map1.add_layer(mh370items[4]) # SatelliteLocation
map1.add_layer(mh370items[1]) # mh370debris

In [9]:
map1.zoom = 6

In [10]:
toolbox_item = gis.content.search("title: Ocean Currents owner:esri_apac", item_type="geoprocessing toolbox", max_items=1)[0]
toolbox_item

<Item title:"Ocean Currents" type:Geoprocessing Toolbox owner:esri_apac>

In [11]:
from arcgis.geoprocessing import import_toolbox
ocean_currents = import_toolbox(toolbox_item)

In [12]:
help(ocean_currents.message_in_a_bottle)

Help on function message_in_a_bottle:

message_in_a_bottle(input_point:arcgis.features.feature.FeatureSet={'geometryType': 'esriGeometryPoint', 'spatialReference': {'wkid': 4326}, 'Fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'Id', 'type': 'esriFieldTypeInteger', 'alias': 'Id'}], 'fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'Id', 'type': 'esriFieldTypeInteger', 'alias': 'Id'}]}, days:float=150, gis=None, future=False) -> arcgis.features.feature.FeatureSet
            
    
    
    Parameters:
    
       input_point: Input Point (FeatureSet). Required parameter. 
    
       days: Days (float). Required parameter. 
    
            gis: Optional, the GIS on which this tool runs. If not specified, the active GIS is used.
    
    
            future: Optional, If True, a future 

In [14]:
from arcgis.features import FeatureSet, Feature

def do_analysis(m, g):
    print("Computing the path that debris would take...")
    
    # Parameter `g` contains the co-ordinates of the clicked location
    
    # Construct a FeatureSet object from the clicked locaiton
    my_feature_set = FeatureSet([Feature(g)])
    
    # Pass the input location as a FeatureSet
    ret = ocean_currents.message_in_a_bottle(my_feature_set, 150)
    
    # Render the resulting FeatureSet on the map using `draw()` method
    map1.draw(ret)
    
# Set the callback function that performs analysis. The `do_analysis` function is called whenever user clicks on the map.
map1.on_click(do_analysis)